In [77]:
# import pandas as pd

# # بارگذاری فایل
# df = pd.read_csv('gen_brg_vib.csv')

# # لیست AssetIDهای مورد نظر
# asset_ids = [9362, 9363, 9364, 9365, 9366, 9367, 9371, 9372]

# # لیست ستون‌های مورد نظر
# columns = ['Value', 'RecordTime', 'RecordDate', 'DateTime', 'TimeStamp']

# # لیست برای نگهداری دیتافریم‌های جداگانه
# dfs = []

# # پردازش هر AssetID به‌صورت جداگانه
# for aid in asset_ids:
#     df_aid = df[df['AssetID'] == aid][columns].copy()
#     df_aid.columns = [f"{col}_{aid}" for col in columns]
#     dfs.append(df_aid.reset_index(drop=True))

# # ترکیب همه دیتافریم‌ها در محور افقی
# df_final = pd.concat(dfs, axis=1)

# # ذخیره فایل نهایی
# df_final.to_csv('flattened_by_assetid.csv', index=False)

# print("✅ فایل نهایی با ستون‌های جداگانه برای هر AssetID ذخیره شد: flattened_by_assetid.csv")


In [78]:
# import pandas as pd

# # بارگذاری فایل خروجی قبلی
# df = pd.read_csv('flattened_by_assetid.csv')

# # لیست AssetIDها
# asset_ids = [9362, 9363, 9364, 9365, 9366, 9367, 9371, 9372]

# # لیست برای نگهداری ردیف‌های نهایی
# final_rows = []

# # تعداد ردیف‌ها
# n = len(df)

# # پیمایش ردیف‌ها
# for i in range(n):
#     row = df.iloc[i]
#     base_values = {}
#     valid = True

#     for aid in asset_ids:
#         base_ts = row[f'TimeStamp_{aid}']
#         if pd.isna(base_ts):
#             valid = False
#             break

#         # تبدیل به عدد برای محاسبه اختلاف
#         try:
#             base_ts = float(base_ts)
#         except:
#             valid = False
#             break

#         # جستجوی نزدیک‌ترین TimeStamp در 5 ردیف بعدی
#         min_diff = float('inf')
#         closest_value = None

#         for j in range(i+1, min(i+6, n)):
#             next_ts = df.iloc[j][f'TimeStamp_{aid}']
#             if pd.isna(next_ts):
#                 continue
#             try:
#                 next_ts = float(next_ts)
#             except:
#                 continue

#             diff = abs(next_ts - base_ts)
#             if diff < 15000 and diff < min_diff:
#                 min_diff = diff
#                 closest_value = df.iloc[j][f'Value_{aid}']

#         if closest_value is not None:
#             base_values[f'Value_{aid}'] = closest_value
#         else:
#             valid = False
#             break

#     if valid:
#         # ذخیره TimeStamp عددی
#         base_values['BaseTimeStamp'] = base_ts
#         # تبدیل به datetime و ذخیره در ستون جداگانه
#         base_values['BaseDateTime'] = pd.to_datetime(base_ts, unit='s')
#         final_rows.append(base_values)

# # ساخت دیتافریم نهایی
# df_final = pd.DataFrame(final_rows)

# # ذخیره فایل نهایی
# df_final.to_csv('matched_values_with_datetime.csv', index=False)

# print("✅ فایل نهایی با ستون datetime قابل استفاده در تایم‌سریز ذخیره شد: matched_values_with_datetime.csv")


In [79]:
# # بارگذاری فایل خروجی قبلی
# df = pd.read_csv('flattened_by_assetid.csv')

# # لیست AssetIDها
# asset_ids = [9362, 9363, 9364, 9365, 9366, 9367, 9371, 9372]

# # لیست برای نگهداری ردیف‌های نهایی
# final_rows = []

# # تعداد ردیف‌ها
# n = len(df)

# # پیمایش ردیف‌ها
# for i in range(n):
#     row = df.iloc[i]
#     base_values = {}
#     valid = True

#     for aid in asset_ids:
#         base_ts = row[f'TimeStamp_{aid}']
#         if pd.isna(base_ts):
#             valid = False
#             break

#         # تبدیل به عدد برای محاسبه اختلاف
#         try:
#             base_ts = float(base_ts)
#         except:
#             valid = False
#             break

#         # جستجوی نزدیک‌ترین TimeStamp در 5 ردیف بعدی
#         min_diff = float('inf')
#         closest_value = None
#         matched_ts = None

#         for j in range(i+1, min(i+6, n)):
#             next_ts = df.iloc[j][f'TimeStamp_{aid}']
#             if pd.isna(next_ts):
#                 continue
#             try:
#                 next_ts = float(next_ts)
#             except:
#                 continue

#             diff = abs(next_ts - base_ts)
#             if diff < 3600 and diff < min_diff:
#                 min_diff = diff
#                 closest_value = df.iloc[j][f'Value_{aid}']
#                 matched_ts = next_ts

#         if closest_value is not None and matched_ts is not None:
#             base_values[f'Value_{aid}'] = closest_value
#             base_values[f'MatchedTimeStamp_{aid}'] = pd.to_datetime(matched_ts, unit='s')
#         else:
#             valid = False
#             break

#     if valid:
#         base_values['BaseTimeStamp'] = base_ts
#         base_values['BaseDateTime'] = pd.to_datetime(base_ts, unit='s')
#         final_rows.append(base_values)

# # ساخت دیتافریم نهایی
# df_final = pd.DataFrame(final_rows)

# # ذخیره فایل نهایی
# df_final.to_csv('matched_values_with_timestamps.csv', index=False)

# print("✅ فایل نهایی با مقادیر Value و TimeStampهای تطبیق‌یافته ذخیره شد: matched_values_with_timestamps.csv")


In [80]:
# import pandas as pd
# import numpy as np

# # مرحله 1: خواندن فایل CSV
# df = pd.read_csv('gen_brg_vib.csv')

# # مرحله 2: استخراج AssetIDهای یکتا
# unique_ids = df['AssetID'].unique().tolist()

# # مرحله 3: ساخت دیتافریم خروجی
# columns = ['TimeStamp'] + [f'AssetID_{uid}' for uid in unique_ids]
# output_df = pd.DataFrame(columns=columns)

# # مرحله 4 تا 8: پردازش تکراری
# while not df.empty and unique_ids:
#     main_id = unique_ids[0]
#     main_subset = df[df['AssetID'] == main_id]

#     if main_subset.empty:
#         # اگر هیچ ردیفی برای این AssetID نبود، حذفش کن و برو سراغ بعدی
#         unique_ids.pop(0)
#         continue

#     # مرحله 5: گرفتن اولین ردیف
#     main_row = main_subset.iloc[0]
#     main_ts = main_row['TimeStamp']
#     main_value = main_row['Value']
#     used_indices = [main_row.name]

#     # مرحله 6: پیدا کردن نزدیک‌ترین TimeStamp برای سایر AssetIDها
#     row_data = {'TimeStamp': main_ts, f'AssetID_{main_id}': main_value}

#     for other_id in unique_ids[1:]:
#         subset = df[df['AssetID'] == other_id].copy()
#         if subset.empty:
#             row_data[f'AssetID_{other_id}'] = np.nan
#             continue

#         subset['ts_diff'] = np.abs(subset['TimeStamp'] - main_ts)
#         close_rows = subset[subset['ts_diff'] <= 1000]

#         if not close_rows.empty:
#             closest_row = close_rows.sort_values('ts_diff').iloc[0]
#             row_data[f'AssetID_{other_id}'] = closest_row['Value']
#             used_indices.append(closest_row.name)
#         else:
#             row_data[f'AssetID_{other_id}'] = np.nan

#     # مرحله 7: حذف ردیف‌های استفاده‌شده
#     df.drop(index=used_indices, inplace=True)

#     # مرحله 8: اضافه کردن به خروجی
#     output_df = pd.concat([output_df, pd.DataFrame([row_data])], ignore_index=True)

# # نمایش خروجی
# print(output_df)


In [81]:
# import pandas as pd
# import numpy as np

# # مرحله 1: خواندن فایل CSV
# df = pd.read_csv('gen_brg_vib.csv')

# # مرحله 2: استخراج AssetIDهای یکتا
# unique_ids = df['AssetID'].unique().tolist()

# # مرحله 3: ساخت دیتافریم خروجی با ستون‌های مورد نظر
# columns = ['TimeStamp'] + [f'AssetID_{uid}' for uid in unique_ids]
# output_df = pd.DataFrame(columns=columns)

# # مرحله 4 تا 8: پردازش تکراری تا خالی شدن دیتافریم اصلی
# while not df.empty and unique_ids:
#     main_id = unique_ids[0]
#     main_subset = df[df['AssetID'] == main_id]

#     if main_subset.empty:
#         unique_ids.pop(0)
#         continue

#     # مرحله 5: گرفتن اولین ردیف از AssetID اصلی
#     main_row = main_subset.iloc[0]
#     main_ts = main_row['TimeStamp']
#     main_value = main_row['Value']
#     used_indices = [main_row.name]

#     # مرحله 6: پیدا کردن نزدیک‌ترین TimeStamp برای سایر AssetIDها
#     row_data = {'TimeStamp': main_ts, f'AssetID_{main_id}': main_value}

#     for other_id in unique_ids[1:]:
#         subset = df[df['AssetID'] == other_id].copy()
#         if subset.empty:
#             row_data[f'AssetID_{other_id}'] = np.nan
#             continue

#         subset['ts_diff'] = np.abs(subset['TimeStamp'] - main_ts)
#         close_rows = subset[subset['ts_diff'] <= 1000]

#         if not close_rows.empty:
#             closest_row = close_rows.sort_values('ts_diff').iloc[0]
#             row_data[f'AssetID_{other_id}'] = closest_row['Value']
#             used_indices.append(closest_row.name)
#         else:
#             row_data[f'AssetID_{other_id}'] = np.nan

#     # مرحله 7: حذف ردیف‌های استفاده‌شده
#     df.drop(index=used_indices, inplace=True)

#     # مرحله 8: اضافه کردن ردیف جدید به خروجی
#     output_df = pd.concat([output_df, pd.DataFrame([row_data])], ignore_index=True)

# # ذخیره خروجی در فایل اکسل
# output_df.to_excel('output.xlsx', index=False)

# print("✅ پردازش کامل شد و خروجی در فایل output.xlsx ذخیره شد.")


In [82]:
import pandas as pd
import numpy as np

# مرحله 1: خواندن فایل CSV
df = pd.read_csv('dsas11_gen_brg_vib_9362_9367_9371_9372.csv')

# مرحله 2: استخراج AssetIDهای یکتا
unique_ids = df['AssetID'].unique().tolist()

# مرحله 3: ساخت دیتافریم خروجی با ستون‌های مورد نظر
columns = ['TimeStamp'] + [f'AssetID_{uid}' for uid in unique_ids]
output_df = pd.DataFrame(columns=columns)

# مرحله 4 تا 8: پردازش تکراری تا خالی شدن دیتافریم اصلی
while not df.empty and unique_ids:
    main_id = unique_ids[0]
    main_subset = df[df['AssetID'] == main_id]

    if main_subset.empty:
        unique_ids.pop(0)
        continue

    # مرحله 5: گرفتن اولین ردیف از AssetID اصلی
    main_row = main_subset.iloc[0]
    main_ts = main_row['TimeStamp']
    main_value = main_row['Value']
    used_indices = [main_row.name]

    # مرحله 6: پیدا کردن نزدیک‌ترین TimeStamp برای سایر AssetIDها
    row_data = {'TimeStamp': main_ts, f'AssetID_{main_id}': main_value}

    for other_id in unique_ids[1:]:
        subset = df[df['AssetID'] == other_id].copy()
        if subset.empty:
            row_data[f'AssetID_{other_id}'] = np.nan
            continue

        subset['ts_diff'] = np.abs(subset['TimeStamp'] - main_ts)
        close_rows = subset[subset['ts_diff'] <= 1800]

        if not close_rows.empty:
            closest_row = close_rows.sort_values('ts_diff').iloc[0]
            row_data[f'AssetID_{other_id}'] = closest_row['Value']
            used_indices.append(closest_row.name)
        else:
            row_data[f'AssetID_{other_id}'] = np.nan

    # مرحله 7: حذف ردیف‌های استفاده‌شده
    df.drop(index=used_indices, inplace=True)

    # مرحله 8: اضافه کردن ردیف جدید به خروجی
    output_df = pd.concat([output_df, pd.DataFrame([row_data])], ignore_index=True)

# مرحله نهایی: تبدیل TimeStamp به تاریخ و زمان و اضافه کردن ستون جدید
output_df['date'] = pd.to_datetime(output_df['TimeStamp'], unit='s')
output_df.drop(columns=['TimeStamp'], inplace=True)
output_df.sort_values(by='date', inplace=True)


# ذخیره خروجی در فایل اکسل
output_df.to_excel('output_gen_brg_vib_9362_9367_9371_9372.xlsx', index=False)

print("✅ پردازش کامل شد، ستون date اضافه شد، و خروجی در فایل output.xlsx ذخیره شد.")


C:\Users\pishva_r\AppData\Local\Temp\ipykernel_14816\347181660.py:52: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([output_df, pd.DataFrame([row_data])], ignore_index=True)


✅ پردازش کامل شد، ستون date اضافه شد، و خروجی در فایل output.xlsx ذخیره شد.
